### Import Libraries

In [13]:
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import from_json

### Configure Session

In [2]:
spark = SparkSession \
    .builder \
    .appName("OpenCommerce") \
    .config("spark.jars", "/ext/lib/postgresql-42.3.4.jar") \
    .getOrCreate()

def get_table(port: int, database: str, table: str):
    return spark.read \
        .format("jdbc") \
        .option("url", f"jdbc:postgresql://host.docker.internal:{port}/{database}?user=postgres") \
        .option("driver", "org.postgresql.Driver") \
        .option("dbtable", table) \
        .load()

### Get Data

In [3]:
actions_df = get_table(9037, "activate_rule_production", "actions")

In [58]:
display(actions_df.select("type").distinct().toPandas())

,type
0,Actions::Discount
1,Actions::AwardPerCountGroup
2,Actions::Purchase
3,Actions::TransactionDiscount
4,Actions::DiscountPerPurchaseGroup
5,Actions::Award
6,Actions::PointAwardPerCountSum
7,Actions::ExternalFulfillment


In [4]:
configurations_rdd = actions_df.select("configuration").rdd.map(lambda x: x[0])
configurations_df = spark.read.json(configurations_rdd)

In [5]:
configurations_df.printSchema()

root
 |-- award_per: string (nullable = true)
 |-- award_type: string (nullable = true)
 |-- count_groups: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- count_by: long (nullable = true)
 |    |    |-- id: string (nullable = true)
 |-- counter_rounding_strategy: string (nullable = true)
 |-- discount_threshold: long (nullable = true)
 |-- discount_type: string (nullable = true)
 |-- discounted_product_counter: struct (nullable = true)
 |    |-- count_by: long (nullable = true)
 |    |-- id: string (nullable = true)
 |-- effect_grouping_configuration: string (nullable = true)
 |-- money_amount: long (nullable = true)
 |-- money_discount: long (nullable = true)
 |-- offer_id: string (nullable = true)
 |-- percent_discount: long (nullable = true)
 |-- point_cost: long (nullable = true)
 |-- point_rounding_strategy: string (nullable = true)
 |-- points: long (nullable = true)
 |-- points_factor: long (nullable = true)
 |-- purchase_amount_threshold: 

In [17]:
actions_df = actions_df.withColumn("configuration", from_json("configuration", "MAP<STRING,STRING>"))

In [64]:
display(actions_df.where("type = 'Actions::Discount'").toPandas())

,id,type,rule_id,configuration,created_at,updated_at,deleted_at,offer_component_type,offer_component_id
0,7112f0b2-a96e-4f2f-9b4f-197e40dfcd88,Actions::Discount,4a85961b-3372-4f6d-b335-30bc32d98a3a,"{'discount_threshold': '0', 'purchase_threshol...",2020-12-15 19:45:51.136853,2020-12-15 19:45:51.136853,NaT,EarningMechanism,ddff5cfc-6368-4e33-8425-a5a6ffbe3cf0
1,a2610e54-02d4-4088-be6f-e1d388677a64,Actions::Discount,be58bb75-cf40-4732-8395-f9e97a7775eb,"{'discount_threshold': '0', 'purchase_threshol...",2020-12-16 05:13:40.044566,2020-12-16 05:13:40.044566,NaT,EarningMechanism,591fcd79-18ae-4de9-ae87-b1ded5ed8a4a
2,1457a50d-c07f-4c8b-b1e6-42a5f859e141,Actions::Discount,ed1d2436-49d6-4f24-9703-1bade0c581c5,"{'discount_threshold': '0', 'purchase_threshol...",2020-12-24 15:54:23.978367,2020-12-24 15:54:23.978367,NaT,EarningMechanism,86925fc0-0f39-4d0e-b2a9-5666b31096b5
3,46af790b-b956-4fda-b1f9-1456b9b5feb7,Actions::Discount,db2af079-28ca-4fb2-a672-571978a8accc,"{'discount_threshold': '0', 'purchase_threshol...",2021-02-12 03:46:12.929642,2021-02-12 03:46:12.929642,NaT,EarningMechanism,321030a8-e2d2-4d1e-98f1-c64cd25ad559
4,dede5397-aa08-44fb-be3c-0e5405b1b260,Actions::Discount,c27850cb-ff9e-47e6-909d-7840555ec454,"{'discount_threshold': '0', 'purchase_threshol...",2021-02-12 03:21:27.543498,2021-02-12 03:47:50.891727,2021-02-12 03:47:50.891717,EarningMechanism,99aade49-541f-4325-a403-d59f769a775c
...,...,...,...,...,...,...,...,...,...
90,35c9b05d-3f4e-4769-b613-0ce06b22ba6c,Actions::Discount,7cef5f14-9c83-4175-b020-d46d621e508d,"{'discount_threshold': '0', 'purchase_threshol...",2022-03-04 13:59:23.026138,2022-03-04 13:59:23.026138,NaT,EarningMechanism,78bb234c-c68c-4ab9-9ed2-24906822e224
91,2c876c93-264c-40e2-9593-31cbd0c61672,Actions::Discount,5d346baf-2e26-4912-8b65-b2a1c8e24240,"{'discount_threshold': '0', 'purchase_threshol...",2022-04-01 10:02:29.299058,2022-04-01 10:02:29.299058,NaT,EarningMechanism,37b19023-3870-4690-b91a-4016b58babe7
92,6d850c18-face-40de-b8ca-9e1f923be1da,Actions::Discount,ffb37dae-ad90-405d-832f-47b1e5e2ef30,"{'discount_threshold': '0', 'purchase_threshol...",2022-04-01 15:36:59.242285,2022-04-01 15:36:59.242285,NaT,EarningMechanism,53ad9c73-f372-417d-85ec-387fe7ea4f91
93,24bfa355-b181-4cb5-bc17-16cf8a0255b5,Actions::Discount,51490b10-23bb-4497-9aac-ae3661645d72,"{'discount_threshold': '0', 'purchase_threshol...",2022-04-01 15:45:11.187406,2022-04-01 15:45:11.187406,NaT,EarningMechanism,9676ca98-91db-47fa-8f60-7b6e6adc50d4


In [55]:
display(configurations_df.where("points_factor > 0").toPandas())

,award_per,award_type,count_groups,counter_rounding_strategy,discount_threshold,discount_type,discounted_product_counter,effect_grouping_configuration,money_amount,money_discount,...,percent_discount,point_cost,point_rounding_strategy,points,points_factor,purchase_amount_threshold,purchase_threshold,voucher_expiration_days,vouchered_rule_id,vouchers
0,None,None,None,down,NaN,None,None,None,NaN,NaN,...,NaN,NaN,down,NaN,10,NaN,NaN,NaN,None,NaN
1,None,None,None,down,NaN,None,None,None,NaN,NaN,...,NaN,NaN,down,NaN,5,NaN,NaN,NaN,None,NaN
2,None,None,None,down,NaN,None,None,None,NaN,NaN,...,NaN,NaN,down,NaN,10,NaN,NaN,NaN,None,NaN
3,None,None,None,down,NaN,None,None,None,NaN,NaN,...,NaN,NaN,down,NaN,10,NaN,NaN,NaN,None,NaN
4,None,None,None,down,NaN,None,None,None,NaN,NaN,...,NaN,NaN,down,NaN,10,NaN,NaN,NaN,None,NaN
5,None,None,None,down,NaN,None,None,None,NaN,NaN,...,NaN,NaN,down,NaN,10,NaN,NaN,NaN,None,NaN
6,None,None,None,down,NaN,None,None,None,NaN,NaN,...,NaN,NaN,down,NaN,10,NaN,NaN,NaN,None,NaN


In [36]:
display

<function IPython.core.display_functions.display(*objs, include=None, exclude=None, metadata=None, transient=None, display_id=None, raw=False, clear=False, **kwargs)>